In [1]:
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio
import matplotlib.pyplot as plt
import time
import numpy as np
import sys
from imutils.object_detection import non_max_suppression
from imutils import paths
import imutils

%matplotlib inline

In [2]:
device = torch.device("cuda")
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [10]:
net = build_ssd('test',300,21).cuda()
#net.load_state_dict(torch.load('weights/ssd300_mAP_77.43_v2.pth'))
net.load_state_dict(torch.load('weights/ssd300_VOC_500.pth'))

In [11]:
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [12]:
def process_image(image):
    height, width = image.shape[:2]
    x = torch.from_numpy(transform(image)[0]).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0).cuda())
    
    y = net(x)
    
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    count = 0
    for i in range (detections.size(2)):
        if detections[0, 15, i, 0] >= 0.4:
            pt = (detections[0, 15, i, 1:] * scale).cpu().numpy()
            cv2.rectangle(image, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 255, 255), 2)
            cv2.putText(image, labelmap[14], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)        
            count+=1
    #print("Total detected people = "+str(count))
    return(image)

In [13]:
def process_video(video):
    cap = cv2.VideoCapture('test_videos/'+video)
    if(_):
        print("Video not found, Try again")
        return
    fps = int(cap.get(5))
    height, width = int(cap.get(3)), int(cap.get(4))
    writer = cv2.VideoWriter("output/"+video, cv2.VideoWriter_fourcc(*"MJPG"), fps,(height, width))
    i=0
    start = time.time()
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            writer.write(process_image(frame))
            cv2.imshow("frame",frame)
            fps = i/(time.time()-start)
            print('FPS = {}'.format(fps), end='\r')
            i=i+1
        else:
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    writer.release()
    cv2.destroyAllWindows()

In [14]:
#process_video('try.mp4')

In [15]:
x = process_image(cv2.resize(cv2.imread('test_images/route.jpg'), (960, 540)))

In [16]:
cv2.imshow("Max",x)
cv2.resizeWindow("Max", 300,300)
cv2.waitKey(0)
cv2.destroyAllWindows()